In [5]:
import reader
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge, LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ParameterGrid

#### Loading the dataset

In [2]:
data=reader.get_all_data()
data.head()

,Property Crime,Violent Crime,Total Crime,Log Property Crime,Log Violent Crime,Log Total Crime,Root Log Property Crime,Root Log Violent Crime,Root Log Total Crime,Mean Income (Household),Median Income (Household),Per Capita Income,Income Standard Deviation (Household),High Bracket Income (Household),Percent High School Dropouts,Percent No Degree,Percent Any Degree
Area_name,,,,,,,,,,,,,,,,,
"Abbeville, SC",307.60,52.200000,359.800000,5.729779,3.967179,5.886334,2.393655,1.991553,2.426142,41.909,33.495,16.772,34.711517,-60.230000,23.1,40.358297,59.641703
"Acadia, LA",569.25,45.000000,614.250000,6.327271,3.545000,6.398918,2.515094,1.873566,2.529226,47.311,33.882,17.704,43.257999,284.213930,31.7,32.914498,67.085502
"Accomack, VA",536.40,70.000000,606.400000,6.280119,4.255036,6.403138,2.505909,2.062554,2.530346,53.054,40.343,22.013,53.847933,358.326667,23.7,35.937033,64.062967
"Ada, ID",1079.00,159.333333,1238.333333,6.976657,5.075962,7.115527,2.641227,2.252961,2.667405,71.190,55.888,27.805,61.384253,318.592294,7.5,47.240193,52.759807
"Adair, IA",44.20,4.600000,48.800000,3.793328,1.602867,3.896170,1.947030,1.245159,1.973473,52.090,43.487,22.897,42.852780,249.226415,7.8,29.580117,70.419883


#### Getting features and label and performing few transformations

In [3]:
from collections import OrderedDict
class Row(object):
    
    def __init__(self):
        self.education_type = None
        self.income_type = None
        self.crime_regularization = None
        self.LinearRegMSE = None
        self.RidgeMSE = None
        self.DecisionTreeMSE = None
        self.KNeighbourMSE = None
        self.SVR_MSE = None
        self.RandomForestMSE = None
        self.BoostingMSE = None

    def toDict(self):
        return {'education_type' : self.education_type,
                'income_type' : self.income_type,
                'crime_regularization': self.crime_regularization,
                'LinearRegMSE':  self.LinearRegMSE,
                'RidgeMSE' : self.RidgeMSE,
                'DecisionTreeMSE' : self.DecisionTreeMSE,
                'KNeighbourMSE': self.KNeighbourMSE,
                'SVR_MSE' : self.SVR_MSE,
                'RandomForestMSE': self.RandomForestMSE,
                'BoostingMSE' :self.BoostingMSE
               }

#### Applying models

In [12]:
import warnings
warnings.filterwarnings("ignore")

param_grid = {"education_type" :["dropout", "degree"], "income_type" :["median", "deviation"], "crime_type":["log", "sqrt_log"]}
result = pd.DataFrame()

for param in list(ParameterGrid(param_grid)):
    row = Row()
    row.education_type = "High School Dropout Percent" if param['education_type'] == 'dropout' else "Percent with Any Degree"
    row.income_type = "Median Income" if param['income_type'] == "median" else "Income Standard Deviation"
    row.crime_regularization = "Square Root of Log" if param['crime_type'] == 'log' else "Fourth Root of Log"
    X = reader.get_features(param['education_type'], param['income_type'])
    y = reader.get_label('total', param['crime_type'])
    y=np.power(y,1/2)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    
    
    ## Scale input data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train.to_numpy())
    X_test = scaler.fit_transform(X_test.to_numpy())
    
    ## Liner Regression
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    y_pred=linreg.predict(X_test)
    row.LinearRegMSE = metrics.mean_squared_error(y_test, y_pred)
    
    ## Ridge Regression
    ridgereg = Ridge(alpha=1.0)
    ridgereg=ridgereg.fit(X_train,y_train)
    y_pred=ridgereg.predict(X_test)
    row.RidgeMSE = metrics.mean_squared_error(y_test, y_pred)
    
    ## Decision Tree
    regr = DecisionTreeRegressor(max_depth=2)
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)
    row.DecisionTreeMSE = metrics.mean_squared_error(y_test, y_pred)
    
    ## Random Forest Tree
    regr = RandomForestRegressor(max_depth=2)
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)
    row.RandomForestMSE = metrics.mean_squared_error(y_test, y_pred)
    
    ### Boosting
    params = {'n_estimators': 100, 'max_depth': 2}
    clf = GradientBoostingRegressor(**params)
    clf.fit(X_train, y_train)
    row.BoostingMSE = metrics.mean_squared_error(y_test, y_pred)
    
    ### KNN
    neigh = KNeighborsRegressor(n_neighbors=3)
    neigh.fit(X_train, y_train) 
    y_pred=neigh.predict(X_test)
    row.KNeighbourMSE = metrics.mean_squared_error(y_test, y_pred)
    
    ### SVR
    svr = SVR(gamma='auto')
    svr = svr.fit(X_train, y_train.values.ravel())
    y_pred=svr.predict(X_test)
    row.SVR_MSE = metrics.mean_squared_error(y_test, y_pred)
   
    
    result = result.append(row.toDict(), ignore_index=True)

result


,BoostingMSE,DecisionTreeMSE,KNeighbourMSE,LinearRegMSE,RandomForestMSE,RidgeMSE,SVR_MSE,crime_regularization,education_type,income_type
0,0.135423,0.142844,0.187286,0.135341,0.135423,0.135341,0.133981,Square Root of Log,High School Dropout Percent,Median Income
1,0.141985,0.150008,0.196002,0.145034,0.141985,0.145033,0.140423,Square Root of Log,High School Dropout Percent,Income Standard Deviation
2,0.138638,0.141787,0.188993,0.134803,0.138638,0.134802,0.142382,Square Root of Log,Percent with Any Degree,Median Income
3,0.136353,0.140847,0.189016,0.137201,0.136353,0.137202,0.136058,Square Root of Log,Percent with Any Degree,Income Standard Deviation
4,0.020787,0.022114,0.029812,0.020799,0.020787,0.020799,0.020618,Fourth Root of Log,High School Dropout Percent,Median Income
5,0.021748,0.021925,0.030625,0.022015,0.021748,0.022014,0.021241,Fourth Root of Log,High School Dropout Percent,Income Standard Deviation
6,0.021496,0.021721,0.029206,0.020920,0.021496,0.020920,0.021868,Fourth Root of Log,Percent with Any Degree,Median Income
7,0.021475,0.021700,0.030082,0.021203,0.021475,0.021203,0.021050,Fourth Root of Log,Percent with Any Degree,Income Standard Deviation


#### SVR Support Vectors

In [14]:
X = reader.get_features("dropout", "median")
y = reader.get_label('total', "sqrt_log")
y=np.power(y,1/2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.to_numpy())
X_test = scaler.fit_transform(X_test.to_numpy())


svr = SVR(gamma='auto')
svr = svr.fit(X_train, y_train.values.ravel())
y_pred=svr.predict(X_test)

print(metrics.mean_squared_error(y_test, y_pred))
supportVectors = len(svr.support_vectors_)
SV=svr.support_vectors_
print("Number of support vectors : ",supportVectors)
print(" ")
print("Support Vectors : \n", SV)

0.13398079549508105
Number of support vectors :  1591
 
Support Vectors : 
 [[ 0.02521343 -0.20740018]
 [ 0.95836928 -0.91747883]
 [ 0.59036416 -1.07106894]
 ...
 [-0.88165634  0.07901964]
 [-0.97365762  0.82290282]
 [ 2.49610499 -0.26986506]]


#### Linear Regression

In [6]:
X = reader.get_features("dropout", "median")
y = reader.get_label('total', "sqrt_log")
y=np.power(y,1/2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.to_numpy())
X_test = scaler.fit_transform(X_test.to_numpy())

linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred=linreg.predict(X_test)
print(metrics.mean_squared_error(y_test, y_pred))

linreg.coef_

0.1348027119497259


array([[-0.0918569 ,  0.05126692]])

array([[0.02535872, 0.04726192]])

#### Linear SVR

In [16]:
from sklearn.svm import LinearSVR

X = reader.get_features("dropout", "median")
y = reader.get_label('total', "sqrt_log")
y=np.power(y,1/2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.to_numpy())
X_test = scaler.fit_transform(X_test.to_numpy())


svr = LinearSVR(C=0.0005)
svr = svr.fit(X_train, y_train.values.ravel())
y_pred=svr.predict(X_test)

print(metrics.mean_squared_error(y_test, y_pred))
SV=svr.coef_
print("Weights : \n", SV)

0.23326286901169918
Number of support vectors :  2
 
Support Vectors : 
 [0.00183921 0.0076809 ]
